In [1]:
import pandas as pd
from tqdm import tqdm
from DB_api import db

In [2]:
header = ['Компания', 'Наименование', 'Регион', 'сайт контрагента', 'НИС', 'Стандарт', 'Диаметр', 'Стенка', 'Сталь',
          'Тип длины', 'Цена наш сайт', 'Цена контрагента', 'Отклонение сайт\сайт', 'Демпинг сайт\сайт']
df = pd.DataFrame(columns=header)
iterator = 0

chelpipe = db.select('chelpipe.*, city.name as region, articles.article as article',
                     'chelpipe, city, articles',
                     'WHERE city_id = city.id AND article_id = articles.id',
                     mod='DISTINCT', mode=0)

In [3]:
for chlpp in tqdm(chelpipe):
    where = 'WHERE '
    if chlpp[2]:
        where += f"standard like '{chlpp[2]}%' AND "
    if chlpp[3]:
        where += f"steel like '{chlpp[3]}%' AND "
    if chlpp[4]:
        where += f"diameter = {chlpp[4]} AND "
    if chlpp[5]:
        where += f"wall = {chlpp[5]} AND "
    if chlpp[6] != 14:
        where += f"city_id = {chlpp[6]} AND "
    where += f"price_min < {chlpp[-6]} AND "
    where = ' '.join(where.split()[:-1])
    res_list = [element for element in db.select('company_price.name, price_min, url, company.name as company',
                                                 'company_price, company',
                                                 where + ' AND company_id = company.id',
                                                 mod='DISTINCT', mode=0)]
    res_list.extend(db.select('company_price.name, price_min, url, company.name as company, city.name as city',
                              'company_price, company, city',
                              'WHERE article_id = ? AND city_id = ? AND price_min < ? AND company_id = company.id AND city.id = city_id',
                              (chlpp[-5], chlpp[6], chlpp[-6]),
                              mod='DISTINCT', mode=0))
    for rl in set(res_list):
        df.loc[iterator] = [rl[3],  # Компания
                            rl[0],  # Наименование
                            rl[-1],  # Регион
                            rl[2],  # сайт контрагента
                            chlpp[-1],  # НИС
                            chlpp[2],  # Стандарт
                            chlpp[4],  # Диаметр
                            chlpp[5],  # Стенка
                            chlpp[3],  # Сталь
                            chlpp[-4],  # Тип длины
                            chlpp[-6],  # Цена наш сайт
                            rl[1],  # Цена контрагента
                            rl[1] - chlpp[-6],  # Отклонение сайт\сайт
                            rl[1] < chlpp[-6]]  # Демпинг сайт\сайт
        iterator += 1

100%|██████████| 98689/98689 [31:59:34<00:00,  1.17s/it]   


In [4]:
df.head()

,Компания,Наименование,Регион,сайт контрагента,НИС,Стандарт,Диаметр,Стенка,Сталь,Тип длины,Цена наш сайт,Цена контрагента,Отклонение сайт\сайт,Демпинг сайт\сайт
0,"""Металлсервис"" ОАО",Трубы ЭСВ 219х8,екатеринбург,https://mc.ru/region/ekaterinburg/metalloproka...,615732,8732-78,219.0,8.0,09Г2С,НК,86500,43900,-42600,True
1,"""Сталепромышленная компания"" АО",Труба профильная,екатеринбург,https://www.spk.ru/catalog/metalloprokat/trubn...,615732,8732-78,219.0,8.0,09Г2С,НК,86500,60325,-26175,True
2,"""Сталепромышленная компания"" АО",Труба бесшовная,екатеринбург,https://www.spk.ru/catalog/metalloprokat/trubn...,615732,8732-78,219.0,8.0,09Г2С,НК,86500,0,-86500,True
3,"""Сталепромышленная компания"" АО",Труба профильная,екатеринбург,https://www.spk.ru/catalog/metalloprokat/trubn...,615732,8732-78,219.0,8.0,09Г2С,НК,86500,61940,-24560,True
4,"""Сталепромышленная компания"" АО",Труба профильная,екатеринбург,https://www.spk.ru/catalog/metalloprokat/trubn...,615732,8732-78,219.0,8.0,09Г2С,НК,86500,58995,-27505,True


In [9]:
mrc = pd.read_excel('output6.xlsx', sheet_name='Отклонения по МРЦ')
with pd.ExcelWriter('output6.xlsx', options={'strings_to_urls': False}) as writer:
    mrc.to_excel(writer, index=False, sheet_name='Отклонения по МРЦ')
    df.to_excel(writer, index=False, sheet_name='Отклонения по сайту')